# Displaying a DataGrid

This example doesn't use the appinator, it only uses the underlying DataGrid widget.

Features shown:
- Display a table
- Set column metadata to control formatting
- Add a click handler for clicking on Cells

A few notes:
- Columns are, by default, Title Cased... so: sepal_length is displayed as Sepal Length

In [ ]:
import logging

import pandas as pd

from nbappinator import ColMd, create_grid

logger = logging.getLogger(__name__)

In [ ]:
grid_data = {"species": ["something"] * 100, "sepal_length": range(100), "some_perc": range(100)}

# To use a more complete dataset:
# data = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

create_grid(pd.DataFrame(grid_data), showindex=True)

In [ ]:
data = pd.DataFrame(grid_data)

In [ ]:
# Same data, but with column metadata definitions

col_md = [
    ColMd(name="species", pinned=True),
    ColMd(name="sepal_length", label="Length of Sepal", format="dec", precision=3),
    ColMd(name="some_perc", label="Some Percentage", format="perc_div", precision=3),
]
grid = create_grid(
    data,
    col_md=col_md,
)
grid

In [ ]:
# With click events

col_md = [
    ColMd(name="species", pinned=True),
    ColMd(name="sepal_length", label="Length of Sepal", format="dec", precision=3),
    ColMd(name="some_perc", hidden=True),
]

click_events = []


def cell_clicked(evt):
    click_events.append(evt)
    print(f"cell clicked {evt}")


grid3 = create_grid(
    data,
    col_md=col_md,
    action=cell_clicked,
)

# Test that handler works
grid3.process_message({"event_type": "cellClicked", "col_clicked": "species", "value_clicked": "test"})
assert len(click_events) == 1, "Click handler should have been called"
print("Click handler test passed")

grid3

In [ ]:
# Use a name that's not a string

col_md = [
    ColMd(name="species"),
    ColMd(name="sepal_length", label="Length of Sepal", format="dec", precision=3),
]
grid_b = create_grid(
    data,
    col_md=col_md,
)
grid_b

In [ ]:
# With multi-select

multi_click_events = []


def cell_clicked_multi(evt):
    multi_click_events.append(evt)
    print(f"cell clicked {evt}")


multiple_grid = create_grid(
    data,
    select_mode="multiple",
    action=cell_clicked_multi,
)

# Test that handler works
multiple_grid.process_message({"event_type": "cellClicked", "col_clicked": "species", "value_clicked": "test"})
assert len(multi_click_events) == 1, "Multi-select click handler should have been called"
print("Multi-select click handler test passed")

display(multiple_grid)

In [ ]:
# Grid without action handler (display only)

display_only_grid = create_grid(
    data,
    select_mode="single",
)
print("Display-only grid created (no click handler)")
display_only_grid